# Thermal Radiation

In [28]:
# Use our tools (you might need to add packages, see file)
using CUDA
device!(2)
# 0 indexed

include("ese156_tools.jl")
using RadiativeTransfer
using RadiativeTransfer.Architectures
using RadiativeTransfer.CrossSection
using ..Architectures: GPU

using JLD2
# using PyPlot
# use float32



---
#### Define file and lat/lon

In [29]:
# To change things up, choose lat/lon of your birthplace (or any place you like!)
file_folder = "files"
# Locate articafts, see https://julialang.github.io/Pkg.jl/v1/artifacts/
# file = joinpath(file_folder, "MERRA300.prod.assim.inst6_3d_ana_Nv.20150613.hdf.nc4")

file = "/home/cchristo/proj_christian/rad_transfer_julia/files/MERRA300.prod.assim.inst6_3d_ana_Nv.20150613.hdf.nc4"
timeIndex = 2 # There is 00, 06, 12 and 18 in UTC, i.e. 6 hourly data stacked together

# What latitude do we want?
myLat = 72.58;
myLon = -38.47;

In [30]:
# Read profile (and generate dry/wet VCDs per layer)
profile = read_atmos_profile(file, myLat, myLon, timeIndex);

ds["T"] = T (540 × 361 × 72 × 4)
  Datatype:    Float32
  Dimensions:  XDim × YDim × Height × TIME
  Attributes:
   _FillValue           = 1.0e15
   long_name            = Air temperature
   standard_name        = air_temperature
   units                = K
   scale_factor         = 1.0
   add_offset           = 0.0
   missing_value        = 1.0e15
   fmissing_value       = 1.0e15
   vmin                 = -1.0e30
   vmax                 = 1.0e30
   valid_range          = Float32[-1.0f30, 1.0f30]
   coordinates          = TIME Height YDim XDim
   fullpath             = /EOSGRID/Data Fields/T



In [4]:
function pressure_filter(var_array::Array, t_array::Array, p_threshold::Real)
    p_thres_inds = findall(var_array -> (var_array > p_threshold), var_array)
    return var_array[p_thres_inds], t_array[p_thres_inds] 
end

pressure_filter (generic function with 1 method)

In [151]:
# length(profile.vcd_h2o)

In [7]:
p_array = profile.p
p_filt_inds = findall(p_array -> (p_array > 15000), p_array)

profile_filt = AtmosphericProfile(profile.lat, 
                                  profile.lon, 
                                  profile.psurf, 
                                  profile.T[p_filt_inds], 
                                  profile.q[p_filt_inds], 
                                  profile.p[p_filt_inds], 
                                  profile.p_levels[p_filt_inds], 
                                  profile.vmr_h2o[p_filt_inds], 
                                  profile.vcd_dry[p_filt_inds], 
                                  profile.vcd_h2o[p_filt_inds])




AtmosphericProfile{Float64}(72.58, -38.47, 68309.4375, [229.61293029785156, 230.0013427734375, 229.46194458007812, 227.2235870361328, 224.24952697753906, 225.36508178710938, 228.47772216796875, 231.8462371826172, 234.6579132080078, 236.8384552001953  …  250.45309448242188, 250.91180419921875, 251.8614501953125, 252.51412963867188, 252.95654296875, 253.18356323242188, 253.27947998046875, 253.2998046875, 253.74810791015625, 250.8155517578125], [5.0486814870964736e-6, 5.692243576049805e-6, 3.073364496231079e-6, 7.543712854385376e-7, 2.703070640563965e-5, 7.987022399902344e-5, 0.00011789798736572266, 0.00016117095947265625, 0.00018334388732910156, 0.0001838207244873047  …  0.0007925306563265622, 0.0008392405579797924, 0.0008917509694583714, 0.0009145797812379897, 0.0009327124571427703, 0.0009442089940421283, 0.0009546990040689707, 0.0009663659147918224, 0.0009724379633553326, 0.0007615452632308006], [16366.15359954144, 19143.82050070841, 22097.244301109065, 25194.41723253252, 28469.4386634

In [8]:
# profile.T = CuArray(profile.T)

plot(t_filt)
# map(maxof, profile.p)
# findmax(profile.p)
# length(profile.p[profile.p .< 10000])
# profile.p

LoadError: [91mUndefVarError: t_filt not defined[39m

---
#### Define HITRAN parameters

In [9]:
# Minimum wavenumber
ν_min  = 400.0
# Maximum wavenumber
ν_max = 2100.0

# define grid
res = 0.01
ν = ν_min:res:ν_max

co2_par = CrossSection.read_hitran(joinpath(file_folder, "hitran_molec_id_2_CO2.tar"), mol=2, iso=1, ν_min=ν_min, ν_max=ν_max);
ch4_par = CrossSection.read_hitran(joinpath(file_folder, "hitran_molec_id_6_CH4.tar"), mol=6, iso=1, ν_min=ν_min, ν_max=ν_max);
h2o_par = CrossSection.read_hitran(joinpath(file_folder, "hitran_molec_id_1_H2O.tar"), mol=1, iso=1, ν_min=ν_min, ν_max=ν_max);

In [9]:
# GPU
co2_voigt   = make_hitran_model(co2_par, Voigt(), wing_cutoff=10, CEF=HumlicekWeidemann32SDErrorFunction(), architecture=CrossSection.GPU())
h2o_voigt   = make_hitran_model(h2o_par, Voigt(), wing_cutoff=10, CEF=HumlicekWeidemann32SDErrorFunction(), architecture=CrossSection.GPU())
ch4_voigt   = make_hitran_model(ch4_par, Voigt(), wing_cutoff=10, CEF=HumlicekWeidemann32SDErrorFunction(), architecture=CrossSection.GPU())

# CPU
# co2_voigt   = make_hitran_model(co2_par, Voigt(), wing_cutoff=10)
# h2o_voigt   = make_hitran_model(h2o_par, Voigt(), wing_cutoff=10)
# ch4_voigt   = make_hitran_model(ch4_par, Voigt(), wing_cutoff=10)



hitran_array = [co2_voigt, h2o_voigt, ch4_voigt];

---
#### Define model resolution and compute all cross sections for profile

In [60]:
# res = 0.01
# ν = ν_min:res:ν_max
# σ_matrix = compute_profile_crossSections(profile, hitran_array , ν);

# Use saved interpolation models

In [10]:
# load interp model 
# interp_model_co2 = load_interpolation_model("/net/fluo/data2/groupMembers/cchristo/interp_models/test/co2_p10_t_5")
interp_model_co2 = @load "/net/fluo/data2/groupMembers/cchristo/interp_models/test/co2_p10_t_5" 
interp_model_co2  = eval(interp_model_co2[1])

interp_model_h2o = @load "/net/fluo/data2/groupMembers/cchristo/interp_models/test/h2o_p10_t_5"
interp_model_h2o = eval(interp_model_h2o[1])

interp_model_ch4 = @load "/net/fluo/data2/groupMembers/cchristo/interp_models/test/ch4_p10_t_5"
interp_model_ch4 = eval(interp_model_ch4[1])

InterpolationModel
  itp: Interpolations.BSplineInterpolation{Float64,3,OffsetArrays.OffsetArray{Float64,3,Array{Float64,3}},BSpline{Cubic{Line{OnGrid}}},Tuple{Base.OneTo{Int64},Base.OneTo{Int64},Base.OneTo{Int64}}}
  mol: Int64 6
  iso: Int64 1
  broadening: Voigt Voigt()
  ν_grid: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  p_grid: StepRange{Int64,Int64}
  t_grid: StepRange{Int64,Int64}
  wing_cutoff: Int64 40
  vmr: Int64 0
  CEF: HumlicekWeidemann32SDErrorFunction HumlicekWeidemann32SDErrorFunction()
  architecture: GPU GPU()


In [11]:
hitran_array_interp = [interp_model_co2, interp_model_h2o, interp_model_ch4];

In [21]:
# interp_model_ch4

In [80]:
typeof(hitran_array_interp[1])

InterpolationModel

In [12]:
σ_matrix = compute_profile_crossSections_interp(profile_filt, hitran_array_interp , ν);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:05


# Reduce dims and Radiative Transfer

In [13]:
# Reduce dimensions, group layers together to get roughly layers of equal pressure difference:
# n_layers = 20
# profile_red, σ_matrix_red = reduce_profile(n_layers, profile_filt, σ_matrix);
profile_red = profile_filt ; 

In [17]:
# i = 2; j = 20;
# plot(1e4./ν,σ_matrix[:,j,i].+1e-90, yscale=:log10,alpha=0.8, lw=2,label="H₂O")
# i = 1;
# plot!(1e4./ν,σ_matrix[:,j,i].+1e-90, yscale=:log10, alpha=0.6,lw=2,label="CO₂")
# i = 3; 
# plot!(1e4./ν,σ_matrix[:,j,i].+1e-90, yscale=:log10, alpha=0.4,lw=2,label="CH₄")
# ylims!(1e-35, 1e-15)
# xlabel!("Wavelength (μm)")
# ylabel!("Cross Section (cm²)")

In [14]:
# Define concentration profile:
nL = length(profile_red.T)
vmr_co2 = zeros(nL) .+ 400e-6
vmr_ch4 = zeros(nL) .+ 2e-6
vmr_h2o = profile_red.vcd_h2o ./ profile_red.vcd_dry
vmrs = [vmr_co2, vmr_h2o, vmr_ch4 ];
@show nL

nL = 32


32

---
### Define Instrument models:

In [126]:
# Define an Instrument, TCCON specs
FWHM = 0.2  # 0.2cm-1 resolution
Δν = 0.05
satellite = KernelInstrument(gaussian_kernel(FWHM, res), collect(minimum(ν)+2FWHM:Δν:maximum(ν)-2FWHM));

### Planck Curve
Here, we need to define the blackbody radiation as a function of wavelength and Temperature. The equation for total outgoing radiance as a function of wavelength wavelength would be
$$B_\lambda(T)\mathrm{d}\lambda = \frac{2 h c^2}{\lambda^5}\frac{1}{e^{\frac{hc}{\lambda k_BT}}-1}\mathrm{d}\lambda\,$$
with $k_B$ being the Boltzmann constant, $h$ the Planck constant, and c the speed of light. Here, $B_\lambda(T)$ is described in terms of the power (W) emitted per unit area of the body, per unit solid angle (sr) that the radiation is measured over, per unit frequency (in SI unit of meters here, we will need to convert to get to nm or $\mu$m). For the sun, we will want to look at the total radiation emitted at the sun's surface, for which we just have to integrate across all solid angles in one hemisphere, effectively mulitplying by $\pi$.  

Similarly, if we are looking at solar irradiance at the top of the Earth's atmosphere, we need to take the distance between the sun and the earth into account. We do so by computing the ratio of the sphere area with radius of sun-earth distance (1AU) and the surface area of the sun. The radius of the sun is about 695,700km. The distance between the centers of sun and earth is 1 Astronomical Unit (AU), i.e. 149,597,870.7km. The ratio of the distances (sun center to sun surface vs sun center to earth surface) is thus (149597870.7-6300)/695700=215.02. Thus, we have to divide the solar irradiance at the sun's surface by 215.02$^2$ to get to the fluxes at the extended distance. The conversion factor is about 2.1629e-05. 
___________
 First, we need to actually define the Planck function. 

In [177]:
# Some constants and the planck function (as radiance!)
# plank functions of wavelength
# function planck(wav, T)
#     h = 6.626e-34
#     c = 299792458
#     k = 1.38e-23
#     c1 = 2h*c^2
#     c2 = h*c/(wav*k*T)
#     intensity = c1/ ( (wav^5)*(exp(c2) - 1.0) )
#     # Convert to W/sr/m^2/µm here directly (it was W/sr/m^2/m)
#     return intensity*1.0e-6
# end


planck (generic function with 1 method)

In [15]:
function planck(wavenum, T)
    wavenum_m = wavenum*1.0e-2
    h = 6.62607004e-34 # m^2/kg/s
    c = 299792458 # m/s
    k = 1.380649e-23 # J/K
    c1 = 2.0*h*(c^2)*(wavenum_m^3)
    c2 = (h*c*wavenum_m)/(k*T)
    intensity = c1/(exp(c2) - 1.0)
    return intensity*1.0e-2
end


planck (generic function with 1 method)

In [195]:
# ν_grid = collect(ν)
# ν_grid_cu = CuArray(ν_grid)
# ν_grid
# planck.(ν_grid, profile.T[1])
# wavenum = ν_grid
# h = 6.62607004e-34 # m^2/kg/s
# c = 299792458 # m/s
# k = 1.380649e-23 # J/K
# c1 = 2.0*h*(c^2)*(wavenum^3)

In [232]:
# wavenum
# @time map(x -> plank(x, 280), ν_grid_cu)
# @time map(x -> plank(x, 280), ν_grid)

In [241]:
# Define a wavelengths grid (equation demands SI unit of meters).
# wl = (1e7*1e-9)./collect(ν[1:100:end]);
# plot(ν, planck.(ν,280), label="Planck Curve",lw=2)
# ylabel!("Radiance (W/sr/m²/μm)")
# xlabel!("Wavelength (μm)")

In [55]:
display("here")

"here"

---
### Define a simplified Forward model:

In [24]:
function forward_model_thermal_gpu(vmrs, σ_matrix, profile, albedo, Tsurf, μ, ν)
    nProfile = length(profile.T)
    nSpec    = size(σ_matrix,1)
    nGas     = length(vmrs)
    
    # wavelength in meter
#     wl = CuArray((1e7*1e-9)./collect(ν))
#     wl = CuArray((1e7*1e-9)./collect(ν)
#     display(wl)
    # Total sum of τ
    ∑τ       = CuArray(zeros(nSpec,nProfile))
#     ∑τ       = zeros(nSpec,nProfile)
    
    # Planck Source function
    S        = CuArray(zeros(nSpec,nProfile))
    
    # Layer boundary up and down fluxes:
    rad_down = CuArray(zeros(nSpec,nProfile+1))
    rad_up   = CuArray(zeros(nSpec,nProfile+1))
    
    # Planck Source per layer:
    display("here0")
#     planck.(wl, profile.T[1])
    display("here00")
    for i=1:nProfile
        S[:,i] = planck.(ν, profile.T[i])
    end
    
    display("here1")
    # sum up total layer τ:
    for i=1:nGas,j=1:nProfile
        ∑τ[:,j] .+= σ_matrix[:,j,i] .* (vmrs[i][j] .* profile.vcd_dry[j])'
    end
    
    
    # Transmission per layer
    T = exp.(-∑τ/μ)
    
    # Atmosphere from top to bottom:
    for i=1:nProfile
        rad_down[:,i+1] = rad_down[:,i].*T[:,i] .+ (1 .-T[:,i]).*S[:,i]
    end
#     display(rad_down)
    # Upward flux at surface:
    rad_up[:,end] = (1-albedo)*planck.(ν,Tsurf) .+ Array(rad_down[:,end])*albedo
    display("past")   
    # Atmosphere from bottom to top
    for i=nProfile:-1:1
        rad_up[:,i] = rad_up[:,i+1].*T[:,i] .+ (1 .-T[:,i]) .* S[:,i]
    end
    
    return T, rad_down, rad_up
end

forward_model_thermal_gpu (generic function with 1 method)

In [57]:
#  planck.(wl, profile.T[1])

In [104]:
# σ_matrix_red_cu[]

In [18]:
σ_matrix_red_cu = CuArray(σ_matrix);
# profile_red_cu = CuArray(profile_red)

In [25]:
@time T, rad_down, rad_up = forward_model_thermal_gpu(vmrs,
                                                     σ_matrix_red_cu,
                                                     profile_red,
                                                     0.05, 280.0, 1.0, ν);

"here0"

"here00"

"here1"

LoadError: [91mCUDA error: an illegal memory access was encountered (code 700, ERROR_ILLEGAL_ADDRESS)[39m

In [22]:
# rad_down
length(ν)

170001

In [189]:
# rad_up
# profile_red.T

In [110]:
@time T, rad_down, rad_up = forward_model_thermal(vmrs,σ_matrix_red, profile_red, 0.05, 280.0, 1.0, ν);

  0.490811 seconds (712 allocations: 671.589 MiB, 1.10% gc time)


## Timing:
# CPU: 1.673095 seconds (2.14 M allocations: 775.430 MiB, 12.45% gc time) 
# GPU: ~1 second

In [96]:
# plot(1e4./ν, prod(T,dims=2),lw=2)
# ylabel!("Total Atmosphere Transmission")
# xlabel!("Wavelength (μm)")


In [110]:
# plot(1e4./ν, Array(rad_up[:,1]))
# ylabel!("Radiance (W/sr/m²/μm)")
# xlabel!("Wavelength (μm)")

In [213]:
# rad_down

In [247]:
# plot(ν, Array(rad_down[:,end]))
# ylabel!("Radiance (W/sr/m²/cm)")
# xlabel!("Wavenumber (μm)")

In [206]:
# heatmap(ν, profile_red.p_levels, rad_down', yflip=true)
# xlabel!("Wavenumber (cm⁻¹)")
# ylabel!("pressure (Pa)")
# title!("Downwelling radiation")

In [20]:
heatmap(ν, profile_red.p_levels, rad_up', yflip=true)
xlabel!("Wavenumber (cm⁻¹)")
ylabel!("pressure (Pa)")
title!("Downwelling radiation")

LoadError: UndefVarError: rad_up not defined

In [21]:
heatmap(ν, profile_red.p, T', yflip=true)
xlabel!("Wavenumber (cm⁻¹)")
ylabel!("Transmission")

LoadError: UndefVarError: T not defined